In [1]:
import os
import nltk
from collections import Counter
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from array import *
import tkinter as tk
from tkinter import *
from sklearn.metrics import roc_curve
from numpy import argmax
from tkinter import messagebox
import shutil
# from sklearn.metrics import accuracy_score


In [2]:
window = tk.Tk()
window.title("Content Based Spam Mail Detection")

#Label 1
l1=tk.Label(window,text="WRITE YOUR MESSAGE HERE",font=("CALIBRI",20))
l1.grid(row=0,column=0)
#Scrolled Text
from tkinter import scrolledtext
txt=scrolledtext.ScrolledText(window,width=30,height=5)
txt.grid(row=1,column=0)

#Label 2
l2=tk.Label(window,text="RESULT",font=("CALIBRI",20))
l2.grid(row=2,column=0)
#TextField 2
t2_var = DoubleVar()
t2=tk.Entry(window,width=40,bd=5,textvariable=t2_var)
t2.grid(row=3,column=0)

def get_wordnet_pos(word):
    #Lemmatization for rootword
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def total_email(data_dir):
    
   
    
    path = [os.path.join(data_dir,f) for f in os.listdir(data_dir)]

    c= len(path)
    total = len(path)



    
    return (total)



def make_datset(data_dir):
    path = [os.path.join(data_dir,f) for f in os.listdir(data_dir)]
    lemma_spam =[]
    lemma_hams = []
    feature_set=[]

    filtered_spam=[]
    filtered_ham=[]
    c= len(path)
    spam=0
    ham=0
    
    for email in path:
        if "spam" in email:

            spam+=1
            f= open(email,encoding="latin-1")
            blob = f.read()
            stop_words = set(stopwords.words('english'))
            word_tokens = word_tokenize(blob)
        

        
            c-=1
            for w in word_tokens: 
                if w not in stop_words: 
                    filtered_spam.append(w) 
  
    
            for i in range(len(filtered_spam)):
                if not filtered_spam[i].isalpha():
                    filtered_spam[i]=""
        
         
            while("" in filtered_spam) : 
                filtered_spam.remove("")
                
            lemmatizer = WordNetLemmatizer()
            for word in range(len(filtered_spam)):
#                 print("[words_spam]",filtered_spam[word])
            
                lemma_spam.append(lemmatizer.lemmatize(filtered_spam[word],get_wordnet_pos(filtered_spam[word])))
   
            dictionary1 = Counter(lemma_spam)
    

        
        if "ham" in email:

            ham+=1
            f= open(email,encoding="latin-1")
            blob = f.read()
            stop_words = set(stopwords.words('english'))
            word_token = word_tokenize(blob)
        

        
            c-=1
            for w in word_token: 
                if w not in stop_words: 
                    filtered_ham.append(w) 
  
    
            for i in range(len(filtered_ham)):
                if not filtered_ham[i].isalpha():
                    filtered_ham[i]=""
        
         
            while("" in filtered_ham) : 
                filtered_ham.remove("") 
                
            lemmatizer = WordNetLemmatizer()
            for word in range(len(filtered_ham)):

                lemma_hams.append(lemmatizer.lemmatize(filtered_ham[word],get_wordnet_pos(filtered_ham[word])))
            
        
        
            dictionary2 = Counter(lemma_hams)


    
        
    return (dictionary1.most_common(),dictionary2.most_common(),spam,ham)
                    
                
        
                    
                
                
                
    
total= total_email("email")
d_spam,d_ham,spam,ham= make_datset("email")



#priors
prior_spam=(spam)/(total)
prior_ham=(ham)/(total)

print("prior_spam",prior_spam)
print("prior_ham",prior_ham)



#Conditional probabilies

def test_class(data_dir,test_spam,test_ham):
    
  
    path = [os.path.join(data_dir,f) for f in os.listdir(data_dir)]
    lemma =[]
    word_tokens=[]
    c= len(path)
    total = len(path)
    filtered_sentence=[]
    for email in path:

            
        f= open(email,encoding="latin-1")
        blob = f.read()

        stop_words = set(stopwords.words('english')) 
        
        word_tokens = word_tokenize(blob)

        
        c-=1
        for w in word_tokens: 
            if w not in stop_words: 
                filtered_sentence.append(w) 
  
        
        
    
        for i in range(len(filtered_sentence)):
            if not filtered_sentence[i].isalpha():
                filtered_sentence[i]=""
 
    
        while("" in filtered_sentence) : 
            filtered_sentence.remove("") 
                
        lemmatizer = WordNetLemmatizer()
        for word in range(len(filtered_sentence)):
        
            lemma.append(lemmatizer.lemmatize(filtered_sentence[word],get_wordnet_pos(filtered_sentence[word])))
        
    
    
            
#         print("test_class",lemma)
#         print(filtered_sentence)
        dictionary = Counter(lemma)
    

    
    
        test_dictionary = dictionary.most_common()
        

        dictionary.clear()

        word_tokens.clear()
        filtered_sentence.clear()
        lemma.clear()
        
        test_ham,test_spam= test_emails(test_dictionary,test_spam,test_ham)
        

    return(test_ham,test_spam)
    
          
    




def test_emails(test_dictionary,test_spam,test_ham):

    test_key=[]
    test_value=[]
    

    for key,value in test_dictionary:
        test_key.append(key)
        test_value.append(value) 


    #Conditional Probabilities
    total =0

    for words,values in d_spam:
    
        total =total+values



    test_words = len(test_dictionary)
    spam_words=len(d_spam)
    ham_words=len(d_ham)
    ham_total =0
    for key,value in d_ham:
        ham_total = ham_total+value


    spam_total =0
    for key,value in d_spam:
        spam_total = spam_total+value


    vocab_spam = len(d_spam)
    vocab_ham = len(d_ham)

    p_spam=[]
    p_ham=[]
    key_test=[]
    val_test=[]
    val_spam= []
    key_spam=[]
    val_ham= array('i',[])

    for key,value in test_dictionary:
        key_test.append(key)
        val_test.append(value)
    
    for key,i in d_spam:
        val_spam.append(i)
        key_spam.append(key)
       



    for key,i in d_ham:
        val_ham.append(i)
    
    found_spam =0

    for key,value in test_dictionary:
        for keys,values in d_spam:
            if key == keys:
                data=(values+1)/(spam_total+vocab_spam)

                p_spam.append(data)
                found_spam+=1
            
        if found_spam == 0:
            data=(0+1)/(spam_total+vocab_spam)

            p_spam.append(data)
        
            


    prob_spam = dict(zip(key_test,p_spam))




    found_ham = 0

    for key,value in test_dictionary:
        for keys,values in d_ham:
        
            if key == keys:
                datas=(values+1)/(ham_total+vocab_ham)

                p_ham.append(datas)
                found_ham+=1
        
        if found_ham ==0:
            datas=(0+1)/(ham_total+vocab_ham)

            p_ham.append(datas)
        
            
#     print(p_ham)

    prob_ham = dict(zip(key_test,p_ham))




    res_spam=1
    for key,value in prob_spam.items():
    
        for keys,values in test_dictionary:
            if key == keys:
                res_spam = res_spam*value**values
                break
               
            else:
                res_spam = res_spam*value
                break
               
        
    

            
    result_spam = prior_spam *res_spam


    res_ham=1
    for key,value in prob_ham.items():
    
        for keys,values in test_dictionary:
            if key == keys:
                res_ham = res_ham*value**values
                break
               
            else:
                res_ham = res_ham*value
                break
               
        
    

            
    result_ham = prior_ham *res_ham

   
  
    if(result_ham>result_spam):

        test_ham+=1

    else:

        test_spam+=1

        
    

    test_dictionary.clear()
    prob_spam.clear()
    prob_ham.clear()

    return (test_ham,test_spam)


test_ham,test_spam = test_class("test_email",0,0)



def accuracy_test(data_dir):
    total_spam_mail=0
    total_ham_mail=0
    spam_acc=0
    ham_acc=0
    accuracy=0
    path = [os.path.join(data_dir,f) for f in os.listdir(data_dir)]
    for email in path:
        if "spam" in email:
            total_spam_mail+=1
        if "ham" in email:
            total_ham_mail+=1
    false_positive = test_spam-total_spam_mail
    false_negative = test_ham- total_ham_mail
    


    accuracy = (test_spam+test_ham)/(test_spam+test_ham+false_positive+false_negative)
    print("accuracy",accuracy)
    recall = test_spam/(test_spam+false_negative)
    print("recall =",recall)
    precision = test_spam/(test_spam+false_positive)
    print("precision=",precision)
    F_measure =(2*precision*recall)/(precision+recall)
    print("F-measure",F_measure)
        
    
    



accuracy_test("test_email")  

def input_test(data_dir):
    path = [os.path.join(data_dir,f) for f in os.listdir(data_dir)]
    lemma =[]
    words=[]
    c= len(path)
    total = len(path)
    filtered_sentence=[]
    for email in path:
            
        f= open(email,encoding="latin-1")
        blob = f.read()
        stop_words = set(stopwords.words('english')) 
        
        word_tokens = word_tokenize(blob)
        
        
        c-=1
        for w in word_tokens: 
            if w not in stop_words: 
                filtered_sentence.append(w) 
  
        
        
    
        for i in range(len(filtered_sentence)):
            if not filtered_sentence[i].isalpha():
                filtered_sentence[i]=""

        while("" in filtered_sentence) : 
            filtered_sentence.remove("")
    
        lemmatizer = WordNetLemmatizer()
        for word in range(len(filtered_sentence)):
        
            lemma.append(lemmatizer.lemmatize(filtered_sentence[word],get_wordnet_pos(filtered_sentence[word])))
        
    
    
            
            
        dictionary = Counter(lemma)
    

    
    
        test_dictionary = dictionary.most_common()
        dictionary.clear()
        words.clear()
        input_test_emails(test_dictionary)
        
    
def input_test_emails(test_dictionary):
    global input_ham
    input_ham =0
    global input_spam
    input_spam =0
    test_key=[]
    test_value=[]
    

    for key,value in test_dictionary:
        test_key.append(key)
        test_value.append(value) 


    #Conditional Probabilities
    total =0

    for words,values in d_spam:
    
        total =total+values



    test_words = len(test_dictionary)
    spam_words=len(d_spam)
    ham_words=len(d_ham)
    ham_total =0
    for key,value in d_ham:
        ham_total = ham_total+value


    spam_total =0
    for key,value in d_spam:
        spam_total = spam_total+value


    vocab_spam = len(d_spam)
    vocab_ham = len(d_ham)

    p_spam=[]
    p_ham=[]
    key_test=[]
    val_test=[]
    val_spam= []
    key_spam=[]
    val_ham= array('i',[])

    for key,value in test_dictionary:
        key_test.append(key)
        val_test.append(value)
    
    for key,i in d_spam:
        val_spam.append(i)
        key_spam.append(key)
       



    for key,i in d_ham:
        val_ham.append(i)
    
    found_spam =0

    for key,value in test_dictionary: 
        for keys,values in d_spam: 
            if key == keys:
                data=(values+1)/(spam_total+vocab_spam)

                p_spam.append(data)
                found_spam+=1
            
        if found_spam == 0:
            data=(0+1)/(spam_total+vocab_spam)

            p_spam.append(data)
        
            


    prob_spam = dict(zip(key_test,p_spam)) 

    print("prob_spam",prob_spam)


    found_ham = 0

    for key,value in test_dictionary:
        for keys,values in d_ham:
        
            if key == keys:
                datas=(values+1)/(ham_total+vocab_ham)

                p_ham.append(datas)
                found_ham+=1
        
        if found_ham ==0:
            datas=(0+1)/(ham_total+vocab_ham)

            p_ham.append(datas)
        


    prob_ham = dict(zip(key_test,p_ham))

    print("prob_ham",prob_ham)


    res_spam=1
    for key,value in prob_spam.items(): 
    
        for keys,values in test_dictionary: 
            if key == keys:
                res_spam = res_spam*value**values 

                    
                break
            else:
                res_spam = res_spam*value
                break
        
    

            
    result_spam = prior_spam *res_spam
    print("result_spam",result_spam)

    res_ham=1
    for key,value in prob_ham.items():
    
        for keys,values in test_dictionary:
            if key == keys:
                res_ham = res_ham*value**values
                break
            else:
                res_ham = res_ham*value
                break
        
    

            
    result_ham = prior_ham *res_ham
    print("result_ham",result_ham)
   
  
    if(result_ham>result_spam):
       
        t2_var.set("Ham")
        input_ham+=1
        
    else:
       
        t2_var.set("Spam")
        input_spam+=1
        

    
          
    
def check_content(data_dir):
    path = [os.path.join(data_dir,f) for f in os.listdir(data_dir)]
    
def get_inputs():
    if len(txt.get("1.0", END))==1:
#         if(txt.get('1.0', END))== 1:
             messagebox.showinfo('Result','Please write a message')
           
    else:
        all_data = [txt.get('1.0', END)]
        print("input",all_data)   
        count = 0
        
           
            
        for data in all_data:
            output = open('input' + str(count) + '.txt', 'w')
            output.write(''.join(data))
            count += 1
            output.close()
            
        for i in range(1):
            shutil.copy2('C:/Users/Jonathan/Desktop/Final_MNB/MNB/input0.txt',
                         'C:/Users/Jonathan/Desktop/Final_MNB/MNB/email/input{}.txt'.format(i))

   

    input_test("input_emails")
    
        
b1=tk.Button(window,text="CHECK",font=("CALIBRI BOLD",15),command=get_inputs)
b1.grid(row=5,column=0)          


def update_directory(data_dir):
    
    if input_ham ==1:
        path = [os.path.join(data_dir,f) for f in os.listdir(data_dir)]
        lemma =[]
        words=[]
        c= len(path)
        total = len(path)
        
        for email in path:
            
            f= open(email,encoding="latin-1")
            blob = f.read()
      
            blob.rstrip("\n")
            
            words=words+blob.split(" ")
            
        words[-1] = words[-1].strip()

            
        dictionary = Counter(words)

        new_dict = dictionary.most_common()
        for key,value in new_dict:
            text=(key,value)
            d_spam.insert(0,text)
           
        
        
    if input_spam ==1:
        path = [os.path.join(data_dir,f) for f in os.listdir(data_dir)]
        lemma =[]
        words=[]
        c= len(path)
        total = len(path)
        
        for email in path:
            
            f= open(email,encoding="latin-1")
            blob = f.read()
            
            blob.rstrip("\n")
            
            words=words+blob.split(" ")
            
        words[-1] = words[-1].strip()

            
        dictionary = Counter(words)

        new_dict = dictionary.most_common()
        for key,value in new_dict:
            text=(key,value)
            d_ham.insert(0,text)
           

        
        
    messagebox.showinfo('Result','Retraining!!!')       

        
    
def retrain():

    email_in_folder = total_email("email")
    start_total= email_in_folder+1
    retrain_total = email_in_folder+2
    if input_ham == 1:
        for i in range(start_total,retrain_total):
            shutil.copy2('C:/Users/Jonathan/Desktop/Final_MNB/MNB/input0.txt',
                         'C:/Users/Jonathan/Desktop/Final_MNB/MNB/email/spam{}.txt'.format(i)) 
        update_directory("input_emails")
        
    if input_spam == 1:
        for i in range(start_total,retrain_total):
            shutil.copy2('C:/Users/Jonathan/Desktop/Final_MNB/MNB/input0.txt',
                         'C:/Users/Jonathan/Desktop/Final_MNB/MNB/email/ham{}.txt'.format(i))  
        update_directory("input_emails")
        

    
    
    
    
b3 = tk.Button(window, text= "Retrain", font=("CALIBRI BOLD", 15), command=retrain)

b3.grid(row=8,column=0)
                   







window.geometry("500x500")
window.mainloop()



    
        
            
        

        




    
     



prior_spam 0.6666666666666666
prior_ham 0.25
accuracy 1.0
recall = 1.0
precision= 1.0
F-measure 1.0
input ['congratulations\n']
prob_spam {'show': 0.01098901098901099}
prob_ham {'show': 0.03571428571428571}
result_spam 0.007326007326007326
result_ham 0.008928571428571428
